In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/RS/sample-data.csv')

In [ ]:
df.head()

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."


In [ ]:
# 500개의 아이템 존재 -> 우리에게 아이템은 바로 메뉴 
df.shape

(500, 2)

In [ ]:
# 아이템에 대한 설명으로 평점이나 다른 지표는 전혀 없다. -> 메뉴에 대한 설명이 얼마나 긴 문장이 될지 고민 필요(메뉴에 대한 상세설명이 부족하면 유사도 측정이 힘듦)
df.iloc[0].description

'Active classic boxers - There\'s a reason why our boxers are a cult favorite - they keep their cool, especially in sticky situations. The quick-drying, lightweight underwear takes up minimal space in a travel pack. An exposed, brushed waistband offers next-to-skin softness, five-panel construction with a traditional boxer back for a classic fit, and a functional fly. Made of 3.7-oz 100% recycled polyester with moisture-wicking performance. Inseam (size M) is 4 1/2". Recyclable through the Common Threads Recycling Program.<br><br><b>Details:</b><ul> <li>"Silky Capilene 1 fabric is ultralight, breathable and quick-to-dry"</li> <li>"Exposed, brushed elastic waistband for comfort"</li> <li>5-panel construction with traditional boxer back</li> <li>"Inseam (size M) is 4 1/2"""</li></ul><br><br><b>Fabric: </b>3.7-oz 100% all-recycled polyester with Gladiodor natural odor control for the garment. Recyclable through the Common Threads Recycling Program<br><br><b>Weight: </b>99 g (3.5 oz)<br><b

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['description'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in df.iterrows():
  similar_indices = cosine_similarities[idx].argsort()[-100:-1] # 유사도가 높은 아이템 순서대로 정렬(index를 리턴)
  similar_items = [(cosine_similarities[idx][i], df['id'][i]) for i in similar_indices]

  results[row['id']] = similar_items[1:] # 자기 자신을 제외한 나머지 아이템들 간의 유사도 저장

# 아이템명 뽑기
def item(id):
  return df.loc[df['id'] == id]['description'].tolist()[0].split(' - ')[0] 

def recommend(item_id, num):
  print("Recommeding " + str(num) + " products simliar to " + item(item_id) + "...")
  print("-" * 20)
  recs = results[item_id][:num]
  for rec in recs:
    print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

recommend(item_id=11, num=5)

Recommeding 5 products simliar to Baby sunshade top...
--------------------
Recommended: Great divider (score:0.02012127852970156)
Recommended: Sticks 'n stones morocco poster (score:0.02022983120002035)
Recommended: Traversing auguille d'entreves (score:0.02121473631499092)
Recommended: Flyfishing the athabasca poster (score:0.02149347273479036)
Recommended: Symmetry w16 poster (score:0.02154414373555964)


In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['description'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in df.iterrows():
  similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
  similar_items = [(cosine_similarities[idx][i], df['id'][i]) for i in similar_indices]

  results[row['id']] = similar_items[1:]

print(len(results))

500


### argsort() 출력값 이해하기
* 행마다 높은 값 기준으로 인덱스 정렬

In [ ]:
x = np.array([[9, 2, 3, 1, 4],
           [4, 5, 6, 5, 7],
           [7, 6, 5, 1, 3],
           [1, 3, 5, 8, 9],
           [5, 7, 9, 1, 2]])

print(x.argsort())

# shape(n, m)일 때, [:-p:-q] 의 경우, 1) 첫 번째 row부터 n-p 번째 row까지, 2) 1번에서 나온 범위(해당 row는 포함x)를 제외한 범위에서 q 번째까지의 row를 제외한 row들 
print(x.argsort()[:-3])
print(x.argsort()[:-3:-2])
print(x.argsort()[:-3:-1])

[[3 1 2 4 0]
 [0 1 3 2 4]
 [3 4 2 1 0]
 [0 1 2 3 4]
 [3 4 0 1 2]]
[[3 1 2 4 0]
 [0 1 3 2 4]]
[[3 4 0 1 2]]
[[3 4 0 1 2]
 [0 1 2 3 4]]
